In [ ]:
!pip install vinorm

In [32]:
import pandas as pd
import IPython
import os
import librosa
import glob 

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
folder_name = "blv AQ"
number_file = len(os.listdir(f"{folder_name}"))
print(f"Number of files: {number_file}")


Number of files: 14


In [55]:
import string
from vinorm import TTSnorm
from evaluate import load

wer_metric = load("wer")

def remove_punctuation(input_string):
    # Tạo một chuỗi chứa tất cả các ký tự dấu chấm câu
    punctuation = string.punctuation
    
    input_string = TTSnorm(input_string, lower = True)
    # Xóa các ký tự dấu chấm câu trong chuỗi đầu vào
    no_punct = "".join([char for char in input_string if char not in punctuation])

    return ' '.join(no_punct.split())


# follow

get file wav -> check duration (<15s) else reject => get line in file predict_wav2vec_blv AQ and foler predict_whisper_blv to create file csv 

In [231]:
index = 8
for i in range(index, index+1):
    print(f"In handling file with index: {i}")
    
    df = pd.DataFrame(columns=['path', 'wav2vec_predict', 'whisper_predict'])
    folder_name_audio = f"split_{folder_name}/{i}"
    predict_wav2vec_path = f"predict_wav2vec_{folder_name}"
    index = i
    paths = []
    whisper_predicts = []
    wav2vec_predicts = []
    durations = []

    
    with open(f"{predict_wav2vec_path}/{index}.txt", "r", encoding="utf-8") as predict_wav2vecs:
        predict_wav2vecs_lines = predict_wav2vecs.readlines()
    
    # len_files = len(os.listdir(folder_name_audio))
    len_files = len(glob.glob(f"{folder_name_audio}/*.wav"))
    print(f" len file: {len_files}")
    for j in range(len_files):
        audio_path = f"{folder_name_audio}/split_{j}.wav"
        predict_whisper_path = f"predict_whisper_{folder_name}/{index}/trans_{j}.txt"
        
        duration = librosa.get_duration(path=audio_path)
        
        if duration > 15:
            # print(f"Audio file: {audio_path}")
            # print("File duration > 15s")
            continue 
        else:
            # Read whisper prediction from file
            
            try:
                with open(predict_whisper_path, "r", encoding="utf-8") as f:
                    whisper_prediction = f.read().strip()
            except FileNotFoundError:
                print(f"File not found: {predict_whisper_path}")
                whisper_prediction = " "
                continue
            
            # Read specific line j from predict_wav2vecs file
            
            wav2vec_prediction = predict_wav2vecs_lines[j].strip() if j < len(predict_wav2vecs_lines) else ""
            
            paths.append(audio_path)
            whisper_predicts.append(whisper_prediction)
            wav2vec_predicts.append(wav2vec_prediction)
            durations.append(duration)
            
            


    # Save DataFrame to CSV file
    df = pd.DataFrame({
            'path': paths,
            'whisper_predict': whisper_predicts,
            'wav2vec_predict': wav2vec_predicts,
            'duration': durations
        })
    df.to_csv(f'map_{index}.csv', index=False)
    
    # Close the file handles
    predict_wav2vecs.close()

In handling file with index: 8
 len file: 674


# Lọc dữ liệu 

In [232]:
# Define a function to compute WER for a row
def compute_wer(row):
    
    wer = wer_metric.compute(predictions =[row['whisper_predict']], references=[row['wav2vec_predict']])
    return wer

In [233]:
df = pd.read_csv(f"map_{index}.csv")

In [234]:
df.describe()

,duration
count,664.000000
mean,7.192073
std,2.099813
min,0.907937
25%,5.537415
50%,6.789569
75%,8.363492
max,14.985941


In [235]:
## remove null row
df = df[df['wav2vec_predict'].notnull()]
df.shape

(664, 4)

In [236]:
df['whisper_predict'] = df['whisper_predict'].apply(remove_punctuation)


text: Vâng, xin chào tất cả mọi người. Chúng ta gặp nhau vào 22h của ngày Chủ nhật.
text: Đơn giản thôi, bởi vì là chúng ta chuẩn bị được chứng kiến trận đấu giữa Chelsea và Manchester United, trận đấu hay nhất của tháng 11, trận đấu người ta quan tâm nhất.
text: Nó có rất nhiều những lý do để quan tâm đúng không ạ? Và rõ ràng ngày hôm nay với những người hâm mộ của Hoàng Anh nói chung.
text: Chứ tôi không nói chỉ riêng những cụm viên của Chelsea hay là Man United thì chúng ta cũng không thể bỏ lỡ trận đấu này được. Đó rõ ràng là...
text: Một trận đấu hấp dẫn cũng như là có điều điều để chờ đợi. Vào thời điểm này, Hoàng Anh cũng đang diễn ra các trận đấu ở vòng đấu này. Trong khi đó,
text: thì đã có một trận đấu bị khoãn vì tiết rơi quá dày ở sân vận động của Burleigh.
text: Trận đấu giữa Burnley và Tottenham Hotspur. Chúng ta cũng đang được chứng kiến trận đấu giữa Man City và West Ham cũng trong một tình trạng tuyết rơi rất dày.
text: và 2 đội đã phải đổi bóng để thi đấu hiệp 2. Rõ r

In [237]:
df.head()

,path,whisper_predict,wav2vec_predict,duration
0,split_blv AQ/8/split_0.wav,vâng xin chào tất cả mọi người chúng ta gặp nh...,vâng xin chào tất cả mọi người chúng ta gặp nh...,5.407710
1,split_blv AQ/8/split_1.wav,đơn giản thôi bởi vì là chúng ta chuẩn bị được...,đơn giản thôi bởi vì chúng ta chuẩn bị được ch...,8.849887
2,split_blv AQ/8/split_2.wav,nó có rất nhiều những lý do để quan tâm đúng k...,nó có rất nhiều những lý do được quan tâm đúng...,7.073923
3,split_blv AQ/8/split_3.wav,chứ tôi không nói chỉ riêng những cụm viên của...,chúng tôi không nói chỉ riêng những cộm viên c...,7.981859
4,split_blv AQ/8/split_4.wav,một trận đấu hấp dẫn cũng như là có điều điều ...,một trận đấu hấp dẫn cũng như là có nhiều điều...,11.174603


In [238]:
df.shape

(664, 4)

In [239]:
# Apply the function to each row and create a new column 'wer'
df['wer'] = df.apply(lambda row: compute_wer(row), axis=1)

In [240]:
df.head()

,path,whisper_predict,wav2vec_predict,duration,wer
0,split_blv AQ/8/split_0.wav,vâng xin chào tất cả mọi người chúng ta gặp nh...,vâng xin chào tất cả mọi người chúng ta gặp nh...,5.407710,0.000000
1,split_blv AQ/8/split_1.wav,đơn giản thôi bởi vì là chúng ta chuẩn bị được...,đơn giản thôi bởi vì chúng ta chuẩn bị được ch...,8.849887,0.194444
2,split_blv AQ/8/split_2.wav,nó có rất nhiều những lý do để quan tâm đúng k...,nó có rất nhiều những lý do được quan tâm đúng...,7.073923,0.071429
3,split_blv AQ/8/split_3.wav,chứ tôi không nói chỉ riêng những cụm viên của...,chúng tôi không nói chỉ riêng những cộm viên c...,7.981859,0.218750
4,split_blv AQ/8/split_4.wav,một trận đấu hấp dẫn cũng như là có điều điều ...,một trận đấu hấp dẫn cũng như là có nhiều điều...,11.174603,0.135135


In [241]:
df.describe()

,duration,wer
count,664.000000,664.000000
mean,7.192073,0.263525
std,2.099813,0.220983
min,0.907937,0.000000
25%,5.537415,0.120000
50%,6.789569,0.212121
75%,8.363492,0.321717
max,14.985941,1.000000


In [242]:
df.shape

(664, 5)

In [243]:
df = df[df.wer <= 0.15 ]

In [244]:
df.reset_index(inplace=True)

In [245]:
df.describe()

,index,duration,wer
count,220.000000,220.000000,220.000000
mean,360.531818,7.008163,0.083554
std,187.089005,2.057444,0.045450
min,0.000000,0.907937,0.000000
25%,207.750000,5.629705,0.047619
50%,387.500000,6.609977,0.092593
75%,503.250000,7.749887,0.120000
max,663.000000,14.676644,0.150000


In [246]:
df.head(10)

,index,path,whisper_predict,wav2vec_predict,duration,wer
0,0,split_blv AQ/8/split_0.wav,vâng xin chào tất cả mọi người chúng ta gặp nh...,vâng xin chào tất cả mọi người chúng ta gặp nh...,5.407710,0.000000
1,2,split_blv AQ/8/split_2.wav,nó có rất nhiều những lý do để quan tâm đúng k...,nó có rất nhiều những lý do được quan tâm đúng...,7.073923,0.071429
2,4,split_blv AQ/8/split_4.wav,một trận đấu hấp dẫn cũng như là có điều điều ...,một trận đấu hấp dẫn cũng như là có nhiều điều...,11.174603,0.135135
3,7,split_blv AQ/8/split_7.wav,và hai đội đã phải đổi bóng để thi đấu hiệp ha...,và hai đội đã phải đổi bóng để thi đấu hiệp ha...,6.714739,0.062500
4,8,split_blv AQ/8/split_8.wav,thời tiết ở nước anh vào thời điểm này đang là...,thời tiết ở nước anh vào thời điểm này đang là...,5.866667,0.000000
5,12,split_blv AQ/8/split_13.wav,trận đấu diễn ra vào lúc hai mươi ba giờ ba mư...,trận đấu diễn ra vào lúc hai mươi ba giờ ba mư...,7.473016,0.103448
6,17,split_blv AQ/8/split_18.wav,và điều đó cho thấy rằng là dù chelsea ngày hô...,và điều đó cho thấy rằng là dù chelsi ngày hôm...,7.842177,0.133333
7,18,split_blv AQ/8/split_19.wav,nhưng khi mà họ thí đấu với manchester united ...,nhưng khi mà họ thi đấu với man sốt ai tứt thì...,9.837642,0.131579
8,20,split_blv AQ/8/split_21.wav,những dòng chat khác nhau đến từ những người b...,ờ những dòng chát khác nhau đến từ những người...,7.433107,0.120000
9,22,split_blv AQ/8/split_23.wav,còn nếu mà để nói rằng tôi trả lời câu hỏi của...,còn nếu mà để nói rằng tôi trả lời câu hỏi của...,5.287982,0.000000


In [247]:
df.drop(['index'], axis=1)

,path,whisper_predict,wav2vec_predict,duration,wer
0,split_blv AQ/8/split_0.wav,vâng xin chào tất cả mọi người chúng ta gặp nh...,vâng xin chào tất cả mọi người chúng ta gặp nh...,5.407710,0.000000
1,split_blv AQ/8/split_2.wav,nó có rất nhiều những lý do để quan tâm đúng k...,nó có rất nhiều những lý do được quan tâm đúng...,7.073923,0.071429
2,split_blv AQ/8/split_4.wav,một trận đấu hấp dẫn cũng như là có điều điều ...,một trận đấu hấp dẫn cũng như là có nhiều điều...,11.174603,0.135135
3,split_blv AQ/8/split_7.wav,và hai đội đã phải đổi bóng để thi đấu hiệp ha...,và hai đội đã phải đổi bóng để thi đấu hiệp ha...,6.714739,0.062500
4,split_blv AQ/8/split_8.wav,thời tiết ở nước anh vào thời điểm này đang là...,thời tiết ở nước anh vào thời điểm này đang là...,5.866667,0.000000
...,...,...,...,...,...
215,split_blv AQ/8/split_664.wav,ngay sau khi chương trình này khép lại cảm ơn ...,ngay sau khi chương trình này khép lại ờ cảm ơ...,14.676644,0.137255
216,split_blv AQ/8/split_669.wav,chúng ta có lẽ là nên kết thúc chương trình nà...,chúng ta có lẽ là kết thúc chương trình này và...,7.443084,0.055556
217,split_blv AQ/8/split_670.wav,đã có tín hiệu từ trận đấu rồi cảm ơn tất cả m...,đã có tín hiệu từ trận đấu rồi cảm ơn tất cả m...,8.650340,0.108108
218,split_blv AQ/8/split_672.wav,cảm ơn các bạn cảm ơn các bạn vì điều đó cảm ơ...,cảm ơn các bạn cảm ơn các bạn vì điều đó cảm ơ...,6.245805,0.000000


In [248]:
print(sum(df.duration)/60)

print(len(df[df.wer == 0]))

25.696598639455804
26


In [249]:
import IPython
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

class TranscriptionTool:
    def __init__(self, df):
        self.df = df
        self.index = 0
        self.output = widgets.Output()
        self.text_whisper = widgets.Textarea(
            value=self.df.loc[self.index, 'whisper_predict'],
            placeholder='Correct transcription here...',
            description='Whisper predict:',
            layout=widgets.Layout(height="30px", width="auto")
        )
        self.text_wav2vec = widgets.Textarea(
            value=self.df.loc[self.index, 'wav2vec_predict'],
            placeholder='Correct transcription here...',
            description='Wav2Vec predict:',
            layout=widgets.Layout(height="30px", width="auto")
        )
        self.button_save_next = widgets.Button(description="Save & Next")
        self.button_next = widgets.Button(description="Next")
        self.button_save_next.on_click(self.on_save_next_button_clicked)
        self.button_next.on_click(self.on_next_button_clicked)
        self.update_output()

    def on_save_next_button_clicked(self, b):
        # Save the current transcription
        self.save_transcription()
        # Load the next transcription
        self.load_next_transcription()

    def on_next_button_clicked(self, b):
        # Load the next transcription without saving
        self.load_next_transcription()

    def save_transcription(self):
        # Update the DataFrame
        self.df.loc[self.index, 'whisper_predict'] = self.text_whisper.value
        self.df.loc[self.index, 'wav2vec_predict'] = self.text_wav2vec.value
        print(self.df.loc[self.index, 'path'])
        with open(self.df.loc[self.index, 'path'].replace('.wav', '.txt'), 'w') as f:
            f.write(self.text_whisper.value)
        
        # Append the transcription to "final_10.txt" with tab separators
        with open(f"final_{index}.csv", 'a') as f:
            f.write(f"{self.df.loc[self.index, 'path']},")
            f.write(f"{self.text_whisper.value},")
            f.write(f"{self.df.loc[self.index, 'duration']}\n")
            
        print("Transcription saved!")

    def load_next_transcription(self):
        self.index += 1
        print(f"index: {self.index}")
        if self.index < len(self.df):
            print(self.df.loc[self.index, 'path'])
            print(self.df.loc[self.index, 'whisper_predict'])
            print(self.df.loc[self.index, 'wav2vec_predict'])
            print(self.df.loc[self.index, 'wer'])
            
            
            # Check if a text file already exists for the current path
            txt_file_path = self.df.loc[self.index, 'path'].replace('.wav', '.txt')
            if os.path.exists(txt_file_path):
                # Load the content from the existing text file and replace self.df.loc[self.index, 'whisper_predict']
                with open(txt_file_path, 'r') as f:
                    existing_content = f.read()
                    self.df.loc[self.index, 'whisper_predict'] = existing_content
            
            self.text_whisper.value = self.df.loc[self.index, 'whisper_predict']
            self.text_wav2vec.value = self.df.loc[self.index, 'wav2vec_predict']
            self.update_output()
        else:
            print("No more data!")

    def update_output(self):
        self.output.clear_output(wait=True)

        with self.output:
            display(IPython.display.Audio(self.df.loc[self.index, 'path']))

    def display(self):
        display(self.output)
        display(self.text_whisper)
        display(self.text_wav2vec)
        display(self.button_save_next)
        display(self.button_next)

# Create the tool and display it
tool = TranscriptionTool(df)
tool.display()


Output()

Textarea(value='vâng xin chào tất cả mọi người chúng ta gặp nhau vào hai mươi hai giờ của ngày chủ nhật', desc…

Textarea(value='vâng xin chào tất cả mọi người chúng ta gặp nhau vào hai mươi hai giờ của ngày chủ nhật', desc…

Button(description='Save & Next', style=ButtonStyle())

Button(description='Next', style=ButtonStyle())

In [227]:
test = pd.read_csv(f"final_{index}.csv", sep=",", names=["path", "label", "duration"])

In [228]:
sum(test.duration)/60

19.46409674981104

In [229]:
test

,path,label,duration
0,split_blv AQ/7/split_0.wav,vâng chào các bạn ngày hôm nay thì rõ ràng là ...,5.258050
1,split_blv AQ/7/split_4.wav,sự chênh lệch về mặt phong độ sự chêch lệch về...,5.397732
2,split_blv AQ/7/split_5.wav,chỉ là một thước đo mang tính tham khảo mà thô...,7.832200
3,split_blv AQ/7/split_9.wav,thì các cầu thủ cũng như các huấn luyện viên đ...,5.776871
4,split_blv AQ/7/split_10.wav,xin chào tất cả mọi người xin chào tất cả các ...,4.948753
...,...,...,...
162,split_blv AQ/7/split_651.wav,đó thì ngày hôm nay thì tôi đã phá được kỷ lục...,11.493878
163,split_blv AQ/7/split_652.wav,tôi rất là cảm ơn tất cả mọi người tôi rất cảm...,6.355556
164,split_blv AQ/7/split_654.wav,ờ ngày hôm nay đặt ra những câu hỏi cho tôi và...,6.555102
165,split_blv AQ/7/split_659.wav,nói rất là nhiều thôi nhưng mà đây là khoản ti...,10.955102


In [256]:
import pandas as pd
import shutil
import os

# Read the new_labels.txt file
df = pd.read_csv("train.txt", sep="|", names=["path", "label"])

# Define the source and destination directories
source_directory = "./"  # Replace with the actual source directory path
destination_directory = "blv_AQ"  # Replace with the actual destination directory path

# Iterate through the DataFrame and copy files
for index, row in df.iterrows():
    source_path = os.path.join(source_directory, row['path'])
    destination_path = os.path.join(destination_directory, row['path'])
    
    print(f"source_path: {source_path}")
    print(f"destination_path: {destination_path}")

    # Create the destination directory if it doesn't exist
    os.makedirs(os.path.dirname(destination_path), exist_ok=True)

    # Copy the file to the destination
    shutil.copyfile(source_path, destination_path)


source_path: ./split_blv AQ/0/split_0.wav
destination_path: blv_AQ/split_blv AQ/0/split_0.wav
source_path: ./split_blv AQ/0/split_1.wav
destination_path: blv_AQ/split_blv AQ/0/split_1.wav
source_path: ./split_blv AQ/0/split_2.wav
destination_path: blv_AQ/split_blv AQ/0/split_2.wav
source_path: ./split_blv AQ/0/split_4.wav
destination_path: blv_AQ/split_blv AQ/0/split_4.wav
source_path: ./split_blv AQ/0/split_10.wav
destination_path: blv_AQ/split_blv AQ/0/split_10.wav
source_path: ./split_blv AQ/0/split_14.wav
destination_path: blv_AQ/split_blv AQ/0/split_14.wav
source_path: ./split_blv AQ/0/split_18.wav
destination_path: blv_AQ/split_blv AQ/0/split_18.wav
source_path: ./split_blv AQ/0/split_21.wav
destination_path: blv_AQ/split_blv AQ/0/split_21.wav
source_path: ./split_blv AQ/0/split_22.wav
destination_path: blv_AQ/split_blv AQ/0/split_22.wav
source_path: ./split_blv AQ/0/split_23.wav
destination_path: blv_AQ/split_blv AQ/0/split_23.wav
source_path: ./split_blv AQ/0/split_24.wav
desti

In [252]:
df = pd.read_csv("train.txt", sep="|", names=["path", "label"])

In [253]:
df


,path,label
0,split_blv AQ/0/split_0.wav,vâng xin chào tất cả anh em đây là một ngày rấ...
1,split_blv AQ/0/split_1.wav,khán giả cũng như là những người đã theo dõi t...
2,split_blv AQ/0/split_2.wav,ngày hôm nay như các bạn đã thấy là chúng ta đ...
3,split_blv AQ/0/split_4.wav,hy vọng rằng là mọi người cũng sẽ cùng ủng hộ ...
4,split_blv AQ/0/split_10.wav,đấy thì hy vọng rằng là mọi người sẽ cùng với ...
...,...,...
2364,blv AQ/13/split_776.wav,là có vấn đề anh nghĩ sao ạ ờ có có thể có thể
2365,blv AQ/13/split_791.wav,bình luận viên quang huy bình luận viên quang ...
2366,blv AQ/13/split_793.wav,chuyện đấy khó mặc dù là bản thân tôi cũng có ...
2367,blv AQ/13/split_800.wav,chúng ta cũng hơi nghĩ quá mọi mọi thứ một chú...
